## 500 bit QPU classification

**This notebook contains the code for inputting data from training sets, applying preprocessing methods from modules, and running data through GB and MLP classifiers**

In [28]:
#imports

from sklearn.model_selection import train_test_split
from modules.classification_model_skeletons import *
from modules.data_preprocessing import *


In [29]:
df = load_data_into_df('data/IBM_Superposition_QRNG_100qubit.txt', ['quantum', 'QPU'])

In [30]:
df_testing = filter_by_label(df, ['binary_number', 'QPU'])

# run below line if classifying by QPU, otherwise comment out 
# (first 40000 lines are QRNG data, next 40000 lines are PRNG data)
df_testing = df_testing[:40000]

df_testing

,binary_number,QPU
0,0001011100100011110100100001101111000011010101...,ibm_brisbane
1,0110101010000001110100000100111101101111110111...,ibm_brisbane
2,1001001010000000001111010010111000100011110000...,ibm_brisbane
3,0101100101110010001001011000011010111011101101...,ibm_brisbane
4,1000101001000000111110011000010111010011111110...,ibm_brisbane
...,...,...
39995,0010010110101100011000101110101101010110000011...,ibm_sherbrooke
39996,1101001111000011101100001011011000010001100011...,ibm_sherbrooke
39997,0010000101001010011001111001100100011011100000...,ibm_sherbrooke
39998,0010110101100001001111101000111100010111111000...,ibm_sherbrooke


In [31]:
# For testing with different input lengths (base length = 100)

df_testing_concatenated = concatenate_data(df_testing, 5)
df_testing_concatenated

,Concatenated_Data,QPU
0,0001011100100011110100100001101111000011010101...,ibm_brisbane
1,0100010001000100101101000000001010001100001011...,ibm_brisbane
2,1011101100100101010010110011110011110100100000...,ibm_brisbane
3,1011011000101001001011011001010000000111100100...,ibm_brisbane
4,0001100011000111101001010110010011110111010100...,ibm_brisbane
...,...,...
7995,0001111100001011000010001100000100111000010000...,ibm_sherbrooke
7996,1110000100010100001110111110011110001100000011...,ibm_sherbrooke
7997,0111100110001101000011000010001101010111110000...,ibm_sherbrooke
7998,1101001110000000011001101101111111101110101001...,ibm_sherbrooke


In [32]:
# Applying preprocessing feature extraction tests:
tests = ['counts']
'''tests = ['counts', 'runs', 'unq_subsq', 'longest_run']'''

# See modules directory for function documentation
df_testing_features = apply_individual_qubit_functions(df_testing_concatenated, tests, 100)
df_testing_features

/Users/shrey/Dev/qc-classifier-experiments/modules/data_preprocessing.py:116: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{function}_qb_{qb}'] = qubitRes
/Users/shrey/Dev/qc-classifier-experiments/modules/data_preprocessing.py:116: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{function}_qb_{qb}'] = qubitRes


,Concatenated_Data,QPU,counts_qb_0,counts_qb_1,counts_qb_2,counts_qb_3,counts_qb_4,counts_qb_5,counts_qb_6,counts_qb_7,...,counts_qb_90,counts_qb_91,counts_qb_92,counts_qb_93,counts_qb_94,counts_qb_95,counts_qb_96,counts_qb_97,counts_qb_98,counts_qb_99
0,0001011100100011110100100001101111000011010101...,ibm_brisbane,3,3,4,2,2,4,1,3,...,2,4,2,3,3,2,2,2,1,4
1,0100010001000100101101000000001010001100001011...,ibm_brisbane,3,3,1,4,2,0,3,2,...,2,4,2,2,4,2,1,2,2,3
2,1011101100100101010010110011110011110100100000...,ibm_brisbane,2,2,3,1,2,3,1,1,...,3,1,3,3,1,2,2,1,2,2
3,1011011000101001001011011001010000000111100100...,ibm_brisbane,3,4,2,0,3,3,1,2,...,2,3,3,3,4,2,4,1,3,3
4,0001100011000111101001010110010011110111010100...,ibm_brisbane,1,3,4,2,4,4,2,4,...,1,2,2,4,2,2,2,2,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,0001111100001011000010001100000100111000010000...,ibm_sherbrooke,5,5,3,3,2,3,1,1,...,2,4,3,2,2,1,3,4,2,3
7996,1110000100010100001110111110011110001100000011...,ibm_sherbrooke,1,0,2,2,2,3,3,2,...,1,0,2,1,1,2,3,3,2,2
7997,0111100110001101000011000010001101010111110000...,ibm_sherbrooke,5,2,3,4,1,3,3,3,...,4,2,3,2,1,3,3,3,1,2
7998,1101001110000000011001101101111111101110101001...,ibm_sherbrooke,3,3,2,2,2,2,2,1,...,3,3,3,4,2,1,4,1,1,3


In [33]:
# For counting individual bits as features: if just using frequency 
# as the only feature, comment out to avoid redundancy: 

'''
df_testing_features = make_bit_features(df_testing_concatenated)
#df_testing_features
'''

df_testing_features = df_testing_features.drop(columns='Concatenated_Data')
df_testing_features

,QPU,counts_qb_0,counts_qb_1,counts_qb_2,counts_qb_3,counts_qb_4,counts_qb_5,counts_qb_6,counts_qb_7,counts_qb_8,...,counts_qb_90,counts_qb_91,counts_qb_92,counts_qb_93,counts_qb_94,counts_qb_95,counts_qb_96,counts_qb_97,counts_qb_98,counts_qb_99
0,ibm_brisbane,3,3,4,2,2,4,1,3,3,...,2,4,2,3,3,2,2,2,1,4
1,ibm_brisbane,3,3,1,4,2,0,3,2,4,...,2,4,2,2,4,2,1,2,2,3
2,ibm_brisbane,2,2,3,1,2,3,1,1,3,...,3,1,3,3,1,2,2,1,2,2
3,ibm_brisbane,3,4,2,0,3,3,1,2,2,...,2,3,3,3,4,2,4,1,3,3
4,ibm_brisbane,1,3,4,2,4,4,2,4,2,...,1,2,2,4,2,2,2,2,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,ibm_sherbrooke,5,5,3,3,2,3,1,1,2,...,2,4,3,2,2,1,3,4,2,3
7996,ibm_sherbrooke,1,0,2,2,2,3,3,2,3,...,1,0,2,1,1,2,3,3,2,2
7997,ibm_sherbrooke,5,2,3,4,1,3,3,3,3,...,4,2,3,2,1,3,3,3,1,2
7998,ibm_sherbrooke,3,3,2,2,2,2,2,1,2,...,3,3,3,4,2,1,4,1,1,3


In [34]:
# Splitting intro training and testing data: 80% and 20% split

X = df_testing_features.drop(columns=df_testing_features.columns[0]).values
y = df_testing_features[df_testing_features.columns[0]].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### GB model

In [35]:
accuracy = sk_learn_gradient_boosting(X_train, y_train, X_test, y_test)
print('GB accuracy: ', accuracy)

fitting model
model fitted, now making predictions on test data
GB accuracy:  0.828125


### Basic MLP model 
**nodes in hidden layer = size of input layer (for consistency)**

In [36]:
num_features = df_testing_features.shape[1] - 1
accuracy = sk_learn_MLP(X_train, y_train, X_test, y_test, num_features)
print('MLP accuracy: ', accuracy)

fitting model
model fitted, now making predictions on test data
MLP accuracy:  0.75625
